In [122]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
tqdm.pandas()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hirunkulphimsiri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [123]:
data = pd.read_csv("data/articles.csv")
data = data.drop(list(data.columns)[2:], axis=1)
data.head()

,title,author_id
0,5 Best Practices For Writing SQL Joins,priyankab14
1,Foundation CSS Dropdown Menu,ishankhandelwals
2,Top 20 Excel Shortcuts That You Need To Know,priyankab14
3,Servlet – Fetching Result,nishatiwari1719
4,Suffix Sum Array,rohit768


In [124]:
all_words: list[str] = []

for i in range(len(data)):
    words: list[str] = data["title"][i].split(" ")
    for word in words:
        all_words.append(word.lower())
        
all_words = list(set(all_words))
# remove stopwords
all_words = [word for word in all_words if word not in stopwords.words('english')]
print("containing", len(all_words), "unique words")

containing 17976 unique words


In [125]:
def bow_encoder(text:str, word_list:list[str]=all_words) -> np.array:
    bow = np.zeros(len(word_list))
    for word in text.split():
        if word in word_list:
            bow[word_list.index(word)] += 1
    return bow

In [126]:
data["embbeding"] = data["title"].progress_apply(lambda x: bow_encoder(x, all_words))

100%|██████████| 34574/34574 [00:56<00:00, 612.15it/s]


In [127]:
def cosine_similarity(a:np.array, b:np.array) -> float:
    return np.dot(a, b) / (np.dot(a,a) **.5) * (np.dot(b,b) ** .5)

In [138]:
def get_similar_articles(text:int, data:pd.DataFrame=data, n:int=5, encoder=bow_encoder) -> pd.DataFrame:
    article_embbeding = encoder(text)
    print("Perform Searching...") 
    data["similarity"] = data["embbeding"].progress_apply(lambda x: cosine_similarity(article_embbeding, x))
    return data.sort_values("similarity", ascending=False).head(n).reset_index(drop=True)

In [139]:
text = "How to generate byte code"
results = get_similar_articles(text)
for idx, result in enumerate(results['title']):
    print(result, "similarity:", round(results['similarity'][idx], 3))

Perform Searching...


  0%|          | 0/34574 [00:00<?, ?it/s]

100%|██████████| 34574/34574 [00:03<00:00, 10105.55it/s]

How to generate byte code file in python ? similarity: 4.243
8085 code to convert binary number to ASCII code similarity: 3.266
Managed code and Unmanaged code in .NET similarity: 2.309
DFA in LEX code which accepts even number of zeros and even number of ones similarity: 2.0
Ayush Jaggi – Geek on the top | Get frustrated by TLEs and WAs, Higher the frustration – better the code similarity: 1.826


In [137]:
text = "python for loop"
results = get_similar_articles(text)
for idx, result in enumerate(results['title']):
    print(result, "[similarity]:", round(results['similarity'][idx], 3))

Perform Searching...


100%|██████████| 34574/34574 [00:03<00:00, 9849.00it/s] 

Using else conditional statement with for loop in python [similarity]: 3.162
foreach() loop vs Stream foreach() vs Parallel Stream foreach() [similarity]: 2.646
Print substring of a given string without using any string function and loop in C [similarity]: 2.236
How to print a number 100 times without using loop and recursion in C? [similarity]: 2.0
Time Complexity where loop variable is incremented by 1, 2, 3, 4 .. [similarity]: 2.0
